# Creating an Extract, Transform and Load process 

### Imports 

In [1]:
import pandas as pd
import mysql.connector
from dotenv import load_dotenv
import os
load_dotenv()

True

### Data

In [10]:
car_sales = pd.read_csv("Data/Cleaned_Car_Sales_Data.csv")

### Creating Connection to Database

In [3]:
conn = mysql.connector.connect(
    host=os.getenv("DB_HOST"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    database=os.getenv("DB_NAME")
)

cur = conn.cursor()

### Reading Raw Data into Database

In [ ]:
cur.execute("""
    CREATE TABLE IF NOT EXISTS raw_data (
        Car_ID VARCHAR(100), Sale_Date DATE, Customer_Name VARCHAR(100), Gender VARCHAR(20), Annual_Income INT,
       Dealer_Name VARCHAR(100), Car_Manufacturer VARCHAR(100), Car_Model VARCHAR(100), Engine_Type VARCHAR(100),
       Transmission VARCHAR(100), Colour VARCHAR(100), Sale_Price VARCHAR(100), Dealer_ID VARCHAR(10), Body_Style VARCHAR(100),
       Customer_Phone_Number VARCHAR(100), Dealer_Region VARCHAR(100), Dealer_State VARCHAR(100))""")

for i, car_sale_row in car_sales.iterrows():
    car_sql = """INSERT INTO raw_data (Car_ID , Sale_Date , Customer_Name , Gender , Annual_Income ,
       Dealer_Name , Car_Manufacturer , Car_Model , Engine_Type ,
       Transmission , Colour , Sale_Price , Dealer_ID , Body_Style ,
       Customer_Phone_Number , Dealer_Region , Dealer_State ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

    car_values = tuple(car_sale_row)

    cur.execute(car_sql, car_values)

conn.commit()


### Creating Tables
- Fact_Tables_Sales
- Dim_Dealer
- Dim_Region
- Dim_Date
- Dim_Car
- Dim_Customer

### Dim_Customer Table

In [15]:
cust_data = car_sales[["Customer_Name", "Gender", "Customer_Phone_Number"]].drop_duplicates().reset_index()

In [105]:
cust_data.to_csv("Data/Customer_Data.csv",index=False)

In [16]:
cust_data["Customer_ID"] = cust_data["index"] + 1

cust_data.drop(columns="index",inplace=True)

In [4]:
cur.execute("""
CREATE TABLE IF NOT EXISTS Dim_Customer (
            Customer_ID INT AUTO_INCREMENT PRIMARY KEY,
            Customer_Name VARCHAR(255),
            Gender VARCHAR(20),
            Customer_Phone_Number VARCHAR(50)
            );""")

cust_sql = """
    INSERT INTO Dim_Customer (Customer_Name, Gender, Customer_Phone_Number)
    SELECT DISTINCT Customer_Name, Gender, Customer_Phone_Number
    FROM raw_data"""

cur.execute(cust_sql)
conn.commit()

### Dim_Car Table

In [98]:
car_data = car_sales[["Car_Manufacturer", "Car_Model", "Engine_Type", "Colour", "Body_Style"]].drop_duplicates().reset_index()
car_data.drop(columns="index",inplace=True)
car_data["Car_ID"] = range(1, len(car_data) + 1)

In [106]:
car_data.to_csv("Data/Car_Data.csv",index=False)

In [5]:
cur.execute("""
    CREATE TABLE IF NOT EXISTS Dim_Car (
                Car_ID INT AUTO_INCREMENT PRIMARY KEY,
                Car_Manufacturer VARCHAR(50),
                Car_Model VARCHAR(50),
                Engine_Type VARCHAR(50),
                Transmission VARCHAR(50),
                Colour VARCHAR(50),
                Body_Style VARCHAR(50)
            );
    """)

car_sql = """
    INSERT INTO Dim_Car (Car_Manufacturer, Car_Model, Engine_Type, Transmission, Colour, Body_Style)
    SELECT DISTINCT Car_Manufacturer, Car_Model, Engine_Type, Transmission, Colour, Body_Style
    FROM raw_data"""

cur.execute(car_sql)
conn.commit()

### Dim_Date

In [61]:
car_sales["Sale_Date"] = pd.to_datetime(car_sales["Sale_Date"])

date_data = car_sales["Sale_Date"].drop_duplicates().reset_index()

date_data.drop(columns="index",inplace=True)

date_data["Year"] = date_data["Sale_Date"].dt.year
date_data["Quarter"] = date_data["Sale_Date"].dt.quarter
date_data["Month"] = date_data["Sale_Date"].dt.month_name()
date_data["Day"] = date_data["Sale_Date"].dt.day_name()
date_data["Date_ID"] = range(1, len(date_data) + 1)

In [107]:
date_data.to_csv("Data/Date_Data.csv",index=False)

In [31]:
cur.execute("""
    CREATE TABLE IF NOT EXISTS Dim_Date (
                Date_ID INT AUTO_INCREMENT PRIMARY KEY,
                Sale_Date DATE,
                Year VARCHAR(50),
                Quarter INT,
                Month VARCHAR(50),
                Day VARCHAR(50)
            );
    """)
date_sql = """
    INSERT INTO Dim_Date (Sale_Date, Year, Quarter, Month, Day)
    SELECT DISTINCT Sale_Date, YEAR(Sale_Date) AS Year , QUARTER(Sale_Date) AS Quarter, MONTHNAME(Sale_Date) AS Month, DAYNAME(Sale_Date) AS Day 
    FROM raw_data"""

cur.execute(date_sql)
conn.commit()


### Dim_Region

In [65]:
region_data = car_sales[["Dealer_Region", "Dealer_State"]].drop_duplicates().reset_index()

region_data.drop(columns="index", inplace=True)

region_data["Region_ID"] = range(1, len(region_data) + 1)

In [108]:
region_data.to_csv("Data/Region_Data.csv",index=False)

In [32]:
cur.execute("""
    CREATE TABLE IF NOT EXISTS Dim_Region (
                Region_ID INT AUTO_INCREMENT PRIMARY KEY,
                Dealer_Region VARCHAR(50),
                Dealer_State VARCHAR(50)
            );
    """)

region_sql = """
    INSERT INTO Dim_Region (Dealer_Region, Dealer_State)
    SELECT DISTINCT Dealer_Region, Dealer_State
    FROM raw_data"""

cur.execute(region_sql)
conn.commit()



### Dim_Dealer 

In [89]:
dealer_data = car_sales[["Dealer_ID", "Dealer_Name"]].drop_duplicates().reset_index()

In [90]:
dealer_data.drop(columns="index",inplace=True)

In [101]:
dealer_data.to_csv("Data/Dealer_Data.csv",index=False)

In [53]:
cur.execute("""
    CREATE TABLE IF NOT EXISTS Dim_Dealer (
            Dealer_ID VARCHAR(10) PRIMARY KEY,
            Dealer_Name VARCHAR(200),
            Region_ID INT,
            FOREIGN KEY (Region_ID) REFERENCES Dim_Region(Region_ID)

            );
    """)

region_sql = """
    INSERT INTO Dim_Dealer (Dealer_ID, Dealer_Name, Region_ID)
    SELECT DISTINCT raw_data.Dealer_ID, MIN(raw_data.Dealer_Name), 
    dim_region.Region_ID FROM raw_data JOIN dim_region ON raw_data.Dealer_Region = dim_region.Dealer_Region 
    GROUP BY raw_data.Dealer_ID
    """

cur.execute(region_sql)
conn.commit()

### Creating Fact Sales Table
- Sale_ID
- Car_ID
- Date_ID
- Customer_ID
- Dealer_ID
- Sale_Price
- Annual Income

In [139]:
dealer_data = pd.read_csv("Data/Dealer_Data.csv")

In [140]:
fact = car_sales.merge(cust_data, on=["Customer_Name","Gender","Customer_Phone_Number"],how="left")
fact = fact.merge(car_data, on=['Car_Manufacturer', 'Car_Model', 'Engine_Type', 'Colour', 'Body_Style'],how="left")
fact = fact.merge(dealer_data, on=["Dealer_ID", "Dealer_Name"],how="left")
fact = fact.merge(date_data, on="Sale_Date", how="outer")
fact = fact.merge(region_data, on=['Dealer_Region', 'Dealer_State', 'Region_ID'],how="left")


In [141]:
fact.drop(columns=['Car_ID_x', 'Sale_Date', 'Customer_Name', 'Gender','Car_Manufacturer', 'Car_Model', 'Engine_Type',
       'Transmission', 'Colour','Body_Style',
       'Customer_Phone_Number', 'Dealer_Region', 'Dealer_State','Region_ID', 'Year', 'Quarter', 'Month', 'Day',"Dealer_Name"],inplace=True)

fact.rename(columns={"Car_ID_y":"Car_ID"},inplace=True)

In [143]:
fact["Sale_ID"] = range(1, len(fact) + 1)

In [145]:
fact.to_csv("Data/Fact_Sales.csv",index=False)